In [3]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset

In [25]:
# 打开对应的数据集
# 先做出来，在封装吧。
# def load_data(path):
temp_path = "./global/temp.npy"
wind_path = "./global/wind.npy"

In [30]:
temp_array = np.load(temp_path)
wind_array = np.load(wind_path)

In [29]:
# temp_array
# del wind_array
# del temp_array

NameError: name 'temp_array' is not defined

In [31]:
temp_array.shape

(17544, 3850, 1)

In [32]:
temp_array.squeeze().shape

(17544, 3850)

In [7]:
# temp_array[:,:,0].shape
temp_array.reshape(-1,1).shape          # 所以，本章的数据处理就是这样的。

(67544400, 1)

In [9]:
# 站点数目
num_sites = temp_array.shape[1]
# 时间数目
num_time_steps = temp_array.shape[0]
num_sites,num_time_steps

(3850, 17544)

In [10]:
# LSTM模型喜欢的数据格式。
# temp_input = temp_array.reshape(-1, num_sites, 1)
# wind_input = wind_array.reshape(-1, num_sites, 1)

In [3]:
# ------- 加载数据 ---------#
## 导包

In [4]:
global_data = np.load('./global/global_data.npy')

In [5]:
# 查看对应数据的维度
global_data.shape

(5848, 4, 9, 3850)

In [16]:
# 对数据进行切片
slice_gdata = global_data[:,:,:,1]
slice_gdata.shape

(5848, 4, 9)

In [8]:
# pd_gdata = pd.DataFrame(slice_gdata)       # 还可以指定对应的列名
# pd_gdata

ValueError: Must pass 2-d input. shape=(5848, 4, 9)

In [20]:
torch_gdata = torch.tensor(slice_gdata)       # 还可以指定对应的列名
# torch_gdata.item()              # 这个是只有一个数时才能使用，一个张量中含有多个值时不能直接打印
print(torch_gdata.shape)
torch_gdata[0].shape

torch.Size([5848, 4, 9])


torch.Size([4, 9])

## 先随便找个模型进行数据的拟合了
### 根据拟合的精度，反过来再看维度的选择

In [22]:
import torch.nn as nn
import math
class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_heads, num_encoder_layers, num_classes):
        super(TransformerModel, self).__init__()
        self.input_dim = input_dim
        self.num_heads = num_heads
        self.num_encoder_layers = num_encoder_layers
        self.num_classes = num_classes

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads,batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_encoder_layers)
        self.positional_encoding = self._generate_positional_encoding(input_dim)
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        x = x + self.positional_encoding[:x.size(0), :]     #
        x = self.transformer_encoder(x)
        out = self.fc(x[:, -1, :])  # 取序列的最后一个时间点作为输出
        return out.view(-1,1)

    def _generate_positional_encoding(self, dim, max_len=4):
        pe = torch.zeros(max_len, dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, dim, 2).float() * (-math.log(10000.0) / dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        return pe.unsqueeze(0)

In [33]:
5848*3

17544

## 测试

In [5]:
temp_data = torch.rand((17544,3850,1))
wind_data = torch.rand((17544,3850,1))
global_data = torch.rand((5848,4,9,3850))

In [102]:
print(temp_data.squeeze(-1).unsqueeze(axis=0).shape)
print(wind_data.squeeze(-1).unsqueeze(axis=0).shape)
print(global_data.reshape(-1, 3850 * 4 * 9).unsqueeze(0).shape)

torch.Size([1, 17544, 3850])
torch.Size([1, 17544, 3850])
torch.Size([1, 5848, 138600])


In [103]:
# temp_data.shape

In [8]:
# 拼接两个数据
wt_data = torch.cat((temp_data,wind_data)).squeeze(-1)  # 压缩最后一个维度
wt_data.shape

torch.Size([35088, 3850])

In [113]:
# global_data.view(-1,3850).shape
global_data_reshaped = global_data.view(-1,3850)
global_data_reshaped.shape

torch.Size([210528, 3850])

In [117]:
# 使用cat方法除了拼接的维度，其他维度都是要相同的
data = torch.cat((wt_data,global_data_reshaped),dim=0)          # 除了融合的维度以外其他维度都需要相同才行
data.shape

torch.Size([245616, 3850])

### 其他拼接张量的方法
- 调用torch.utils.data.TensorDataset

In [15]:
print(wt_data.shape)
print(global_data.view(-1,3850).shape)
# TensorDataSet默认是按
dataset_train = TensorDataset(wt_data.permute(1,0),global_data.view(-1,3850).permute(1,0))

torch.Size([35088, 3850])
torch.Size([210528, 3850])
